In [ ]:

# Delete the endpoint
service.delete()
print(f"✅ Deleted existing endpoint: {service_name}")


In [16]:
from azureml.core import Workspace, Model, Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

# ✅ Load Azure ML Workspace
ws = Workspace.from_config()

# ✅ Verify ACR is correctly assigned
acr_name = "mlopssimpleacr"
acr_scope = f"/subscriptions/a9e6b3ce-7394-4a87-9482-c314c87e5743/resourceGroups/mlops-simple-flow/providers/Microsoft.ContainerRegistry/registries/{acr_name}"
print(f"Using ACR: {acr_scope}")

# ✅ Ensure the model is registered
model_name = "catboost-thrombosis-predictor"
model = Model(ws, name=model_name)
print(f"✅ Model {model_name} found in registry.")

# ✅ Create environment for deployment
env = Environment(name="catboost-env")
env.python.conda_dependencies.add_pip_package("catboost")
env.python.conda_dependencies.add_pip_package("azureml-defaults")

# ✅ Create inference configuration
inference_config = InferenceConfig(
    entry_script="score.py",  # Your custom scoring script
    environment=env
)

# ✅ Define deployment configuration (1 CPU, 1GB RAM)
aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1, memory_gb=1, 
    tags={"framework": "catboost", "task": "thrombosis-prediction"},
    description="Predict thrombotic events using CatBoost",
    auth_enabled=True,  # Enables authentication for security
    enable_app_insights=True,  # Enables logging & monitoring
)

# ✅ Deploy model as a web service
service_name = "catboost-endpoint"

# 🔥 First, delete existing deployment if it exists
try:
    existing_service = Webservice(ws, name=service_name)
    print(f"🚨 Existing service '{service_name}' found. Deleting...")
    existing_service.delete()
    print(f"✅ Deleted old service '{service_name}'.")
except:
    print(f"🔹 No existing service '{service_name}' found. Proceeding with deployment.")

# ✅ Deploy the model
print(f"🚀 Deploying model '{model_name}' to Azure Container Instances (ACI)...")
service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[model],
    inference_config=inference_config,
    deployment_config=aci_config
)

# ✅ Wait for deployment to complete
service.wait_for_deployment(show_output=True)

# ✅ Print endpoint URL
print(f"✅ Model deployed successfully! Endpoint URL: {service.scoring_uri}")


Using ACR: /subscriptions/a9e6b3ce-7394-4a87-9482-c314c87e5743/resourceGroups/mlops-simple-flow/providers/Microsoft.ContainerRegistry/registries/mlopssimpleacr
✅ Model catboost-thrombosis-predictor found in registry.
🚨 Existing service 'catboost-endpoint' found. Deleting...
Running
2025-01-31 22:40:49+00:00 Deleting service.
2025-01-31 22:40:49+00:00 Deleting service entity.
Succeeded
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Failed

/tmp/ipykernel_4605/177329363.py:52: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(
Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: 99139f0d-beef-41e6-8591-31a30143f9b1
Current sub-operation type not known, more logs unavailable.
Error:
{
  "code": "InaccessibleImage",
  "statusCode": 400,
  "message": "ACI Service request failed. Reason: The image 'mlopssimpleacr.azurecr.io/azureml/azureml_ad26baf1abec22955cd

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: 99139f0d-beef-41e6-8591-31a30143f9b1
Current sub-operation type not known, more logs unavailable.
Error:
{
  "code": "InaccessibleImage",
  "statusCode": 400,
  "message": "ACI Service request failed. Reason: The image 'mlopssimpleacr.azurecr.io/azureml/azureml_ad26baf1abec22955cdd5679dcacaa63' in container group 'catboost-endpoint-9Bc_QrZu9UG9fIJRTYrtoQ' is not accessible. Please check the image and registry credential.. Refer to https://docs.microsoft.com/azure/container-registry/container-registry-authentication#admin-account and make sure Admin user is enabled for your container registry."
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Unhealthy\nOperation ID: 99139f0d-beef-41e6-8591-31a30143f9b1\nCurren

In [12]:
from azureml.core import Model, Environment, Workspace
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

# ✅ Connect to Azure ML Workspace
ws = Workspace.from_config()

# ✅ Load the registered model
model = Model(ws, name="catboost-thrombosis-predictor")

# ✅ Define environment for deployment
env = Environment(name="catboost-env")
env.python.conda_dependencies.add_pip_package("catboost")
env.python.conda_dependencies.add_pip_package("azureml-defaults")

# ✅ Create inference configuration
inference_config = InferenceConfig(
    entry_script="score.py",
    environment=env
)

# ✅ Define deployment configuration (1 CPU, 1GB RAM)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# ✅ Deploy model as a web service
service = Model.deploy(
    workspace=ws,
    name="catboost-endpoint",
    models=[model],
    inference_config=inference_config,
    deployment_config=aci_config
)

service.wait_for_deployment(show_output=True)
print(f"✅ Model deployed successfully! Endpoint URL: {service.scoring_uri}")


/tmp/ipykernel_4605/967373416.py:26: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(
Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: 26f40595-eb69-409a-97d3-cc9b4681e2bf
Current sub-operation type not known, more logs unavailable.
Error:
{
  "code": "InaccessibleImage",
  "statusCode": 400,
  "message": "ACI Service request failed. Reason: The image 'mlopssimpleacr.azurecr.io/azureml/azureml_ad26baf1abec22955cd

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Failed

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: 26f40595-eb69-409a-97d3-cc9b4681e2bf
Current sub-operation type not known, more logs unavailable.
Error:
{
  "code": "InaccessibleImage",
  "statusCode": 400,
  "message": "ACI Service request failed. Reason: The image 'mlopssimpleacr.azurecr.io/azureml/azureml_ad26baf1abec22955cdd5679dcacaa63' in container group 'catboost-endpoint-9Bc_QrZu9UG9fIJRTYrtoQ' is not accessible. Please check the image and registry credential.. Refer to https://docs.microsoft.com/azure/container-registry/container-registry-authentication#admin-account and make sure Admin user is enabled for your container registry."
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Unhealthy\nOperation ID: 26f40595-eb69-409a-97d3-cc9b4681e2bf\nCurren

Test Request

In [ ]:
import requests
import json

# Replace the endpoint-url with the actual URL from Azure ML
endpoint_url = "https://<your-endpoint-url>/score"

# Sample test data (Replace with actual feature values)
test_data = {
    "data": [
        {"gender": 1, "ever_married": 1, "work_type": 2, "Residence_type": 1, "smoking_status": 2, 
         "age": 50.0, "avg_glucose_level": 100.0, "bmi": 25.0}
    ]
}

# Send request
response = requests.post(endpoint_url, json=test_data)

# Print the response
print("Predictions:", response.json())
